In [4]:
""""""
from selenium import webdriver
import time
import pymongo

class JdSpider:
    def __init__(self):
        #设置无界面
        self.options=webdriver.ChromeOptions()
        self.options.add_argument("--headless")
        self.driver=webdriver.Chrome(options=self.options)
        self.driver.get(url='https://www.jd.com')
        #搜索框发送：相关字，点击搜索按钮
        self.driver.find_element_by_xpath('//*[@id="key"]').send_keys("爬虫书")
        self.driver.find_element_by_xpath('//*[@id="search"]/div/div[2]/button').click()
        #给一些元素加载的时间
        time.sleep(1)

        #创建mongodb
        self.conn=pymongo.MongoClient("localhost",27017)
        self.db=self.conn['JD']
        self.myset=self.db["jdset"]

    def parse_html(self):
        """具体的去提取数据"""
        #先把鼠标滚动到最下面，让所有的商品显示出来
        self.driver.execute_script(
            'window.scrollTo(0,document.body.scrollHeight)'
        )
        time.sleep(3)
        #具体去提取数据

        lli_list=self.driver.find_elements_by_xpath('//*[@id="J_goodsList"]/ul/li')
        for li in lli_list:
            try:
                item={}
                item['price']=li.find_element_by_xpath('.//div[@class="p-price"]/strong').text.strip()
                item['name']=li.find_element_by_xpath('.//div[@class="p-name"]/a/em').text.strip()
                item['commit']=li.find_element_by_xpath('.//div[@class="p-commit"]/strong').text.strip()
                item['shop']=li.find_element_by_xpath('.//div[@class="p-shopnum"]/a').text.strip()
                # print(li.text)
                print(item)
                self.myset.insert_one(item)
                print("*"*50)
            except Exception as e:
                print(e)
    
    def run(self):
        while True:
            self.parse_html()
            #如果没有找到点不动的下一页按钮，就是说明不是最后一页
            if self.driver.page_source.find('pn-next disabled')==-1:
                self.driver.find_element_by_xpath('//*[@id="J_bottomPage"]/span[1]/a[9]').click()
                time.sleep(1)
            
            else:
                self.driver.quit()
                break


if __name__ == "__main__":
    
    spider=JdSpider()  #实例化对象的时候执行__init__
    spider.run()


{'price': '￥73.00', 'name': 'Python编程从入门到精通 计算机电脑编程入门自学零基础籍网络 pathon编程从入门到实践', 'commit': '0条评价', 'shop': '争渡文化图书专营店'}
**************************************************
{'price': '￥137.00', 'name': 'Python3网络爬虫开发实战 第2版', 'commit': '50万+条评价', 'shop': '人民邮电出版社'}
**************************************************
{'price': '￥95.70', 'name': 'Python从入门到项目实践（全彩版）PyCharm详解，热门游戏、、数据分析、web和AI开发', 'commit': '20万+条评价', 'shop': '明日科技京东自营旗舰店'}
**************************************************
{'price': '￥19.80', 'name': '【扫码赠视频课程】2021年新版python编程从入门到精通计算机零基础自学python编程从入门到实战编程语言程序精通教程程序设计开发书籍', 'commit': '73条评价', 'shop': '聚德实达图书专营店'}
**************************************************
{'price': '￥29.50', 'name': '正版包邮 从零开始学Python网络爬虫 罗攀 蒋仟Python零基础语法入门 |6821955', 'commit': '37条评价', 'shop': '互动创新图书专营店'}
**************************************************
{'price': '￥93.90', 'name': 'Python网络爬虫从入门到实践（Python3 全彩版）自动化数据采集，入门知识，项目实践，海量资源，技术答疑', 'commit': '20万+条评价', 'shop': '明日科技京东自营旗舰店'}
*****************

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[@class="p-shopnum"]/a"}
  (Session info: headless chrome=100.0.4896.127)
